In [2]:
from pymongo import MongoClient
from bson.objectid import ObjectId
from bson import json_util
from time import time
import datetime as dt
from dateutil.relativedelta import relativedelta
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz, process
import requests
from urllib.parse import unquote
import unidecode
import re
import requests
from bs4 import BeautifulSoup
import json
import pickle

In [3]:
class JSONEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, ObjectId):
            return str(o)
        return json.JSONEncoder.default(self, o)

In [4]:
client=MongoClient()
colombia=client["colombia_udea"]
openalex=client["openalex"]

In [5]:
def empty_subjects():
    return {
        "updated" : [],
        "names" : [],
        "abbreviations" : [],
        "descriptions" : [],
        "external_ids" : [],
        "external_urls" : [],
        "level" : None,
        "relations" : []
    }

In [17]:
inserted_concepts=[]
inserted_concepts_ids_tuples=[]

In [18]:
for sub in openalex["concepts"].find():
    #print(sub)
    if sub["id"] in inserted_concepts:
        continue
    entry=empty_subjects()
    entry["updated"]=[{"source":"openalex","time":int(time())}]
    sources_inserted_ids=[]
    entry["external_ids"].append({"source":"openalex","id":sub["id"]})
    sources_inserted_ids.append("openalex")
    for source,idx in sub["ids"].items():
        if source in sources_inserted_ids:
            continue
        entry["external_ids"].append({"source":source,"id":idx})
        sources_inserted_ids.append(source)
    entry["level"]=sub["level"]
    entry["names"].append({"name":sub["display_name"],"lang":"en"})
    inserted_lang_names=["en"]
    if sub["international"]:
        if sub["international"]["display_name"]:
            for lang,name in sub["international"]["display_name"].items():
                if lang in inserted_lang_names:
                    continue
                entry["names"].append({"name":name,"lang":lang})
                inserted_lang_names.append(lang)
    if sub["description"]:
        entry["descriptions"].append({"description":sub["description"],"lang":"en"})
    if sub["wikidata"]:
        entry["external_urls"].append({"source":"wikidata","url":sub["wikidata"]})
    if sub["image_url"]:
        entry["external_urls"].append({"source":"image","url":sub["image_url"]})
        
    response=colombia["subjects"].insert_one(entry)
    inserted_concepts.append(sub["id"])
    inserted_concepts_ids_tuples.append((response.inserted_id,sub["id"]))
    #print(entry)
    #break

In [24]:
#fix relations on subjects collection
for sub in openalex["concepts"].find():
    relations=[]
    for rel in sub["related_concepts"]:
        sub_db=colombia["subjects"].find_one({"external_ids.id":rel["id"]})
        if sub_db:
            rel_entry={
                "id":sub_db["_id"],
                "names":sub_db["names"],
                "level":sub_db["level"]
            }
            relations.append(rel_entry)
        else:
            print("Could not find related concept in colombia db")
    for rel in sub["ancestors"]:
        sub_db=colombia["subjects"].find_one({"external_ids.id":rel["id"]})
        if sub_db:
            rel_entry={
                "id":sub_db["_id"],
                "names":sub_db["names"],
                "level":sub_db["level"]
            }
            relations.append(rel_entry)
        else:
            print("Could not find related concept in colombia db")
    if len(relations)>0:
        pass
        #print(relations)
        #break
        #colombia["subjects"].update_one({"external_ids.id":sub["id"]},{"$set":{"relations":relations}})

In [ ]:
#insert mesh subjects
descriptors=list(client["openalexco"]["works"].distinct("mesh.descriptor_ui"))
inserted=[]
for idx in descriptors:
    db_openalex=client["openalexco"]["works"].find_one({"mesh.descriptor_ui":idx})
    mesh_openalex=None
    if db_openalex:
        already=False
        for mesh in db_openalex["mesh"]:
            if mesh["descriptor_ui"] == idx:
                if mesh["descriptor_ui"] in inserted:
                    already=True
                else:
                    mesh_openalex=mesh
                break
        if already:
            continue
        else:
            inserted.append(idx)
            entry=empty_subjects()
            entry["updated"]=[{"source":"openalex","time":int(time())}]
            entry ["names"].append({"name":mesh_openalex["descriptor_name"],"lang":"en"})
            entry["external_ids"].append({"source":"mesh","id":idx})